### Importations et configuration du chemin racine du projet

In [1]:
%run ./notebook_2018_extract.ipynb
import pandas as pd  # pour la manipulation de DataFrames
import json           # pour lire et écrire des fichiers JSON
import os             # pour gérer les chemins et interactions système
import yaml           # pour lire et écrire des fichiers YAML
from pathlib import Path  # pour manipuler les chemins de fichiers de manière portable
from typing import Dict   # pour typer les dictionnaires dans les fonctions

# Définir le chemin racine du projet
# Ici on prend le dossier parent du répertoire courant
PROJECT_ROOT = Path.cwd().parent

# Afficher le chemin racine pour vérification
PROJECT_ROOT


PosixPath('/home/kaouter/Bureau/projects/briefs_simplon/brief2/Brief-2-ETL-de-donn-es-footballistiques-Wickets-Sprinters')

### Fonctions utilisées pour la transformation des données

In [2]:
def fct_load_config(config_filename: str = "config.yaml") -> dict:
    """
    Objectif :
        Charger les paramètres de configuration depuis un fichier YAML.
    Paramètres :
        config_filename (str) : Chemin relatif ou absolu du fichier YAML.
    Retour :
        dict : Paramètres de configuration.
    """
    config_path = Path(config_filename)

    # Résoudre le chemin relatif depuis la racine du projet
    if not config_path.is_absolute():
        project_root = Path(__file__).resolve().parents[1]
        config_path = project_root / config_path

    if not config_path.exists():
        raise FileNotFoundError(f"Fichier de configuration introuvable : {config_path}")

    # Charger le fichier YAML
    with open(config_path, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)

    return config


def fct_add_prefix_to_df(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    """
    Objectif :
        Ajouter un préfixe à toutes les colonnes d'un DataFrame.
    Paramètres :
        df (pd.DataFrame) : DataFrame d'entrée.
        prefix (str) : Préfixe à ajouter.
    Retour :
        pd.DataFrame : DataFrame avec les noms de colonnes modifiés.
    """
    for col in df.columns:
        df.rename(columns={col: f"{prefix}_{col}"}, inplace=True)
    return df


def fct_capitalize_string_columns(df: pd.DataFrame, cols: list = None) -> pd.DataFrame:
    """
    Objectif :
        Supprimer les espaces superflus et mettre la première lettre en majuscule,
        le reste en minuscules, uniquement pour les valeurs de type str.
    
    Paramètres :
        df (pd.DataFrame) : DataFrame d'entrée.
        cols (list, optionnel) : Liste des colonnes à nettoyer.
    
    Retour :
        pd.DataFrame : DataFrame avec les colonnes de chaînes nettoyées.
    """
    # Vérifier que des colonnes ont été fournies
    if not cols:
        print("[INFO] Aucune colonne spécifiée pour capitalize_string_columns. Le DataFrame reste inchangé.")
        return df
    
    # Traiter chaque colonne spécifiée
    for col in cols:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype("string")
                .str.strip()
                .str.capitalize()
            )
    return df



def fct_upper_string_columns(df: pd.DataFrame, cols: list = None) -> pd.DataFrame:
    """
    Objectif :
        Supprimer les espaces superflus et mettre toutes les lettres en majuscules,
        uniquement pour les valeurs de type str.
    
    Paramètres :
        df (pd.DataFrame) : DataFrame d'entrée.
        cols (list, optionnel) : Liste des colonnes à nettoyer. 
    Retour :
        pd.DataFrame : DataFrame avec les colonnes de chaînes nettoyées.
    """
    # Vérifier que des colonnes ont été fournies
    if not cols:
        print("[INFO] Aucune colonne spécifiée pour clean_string_columns_upper. Le DataFrame reste inchangé.")
        return df
    
    # Traiter chaque colonne spécifiée
    for col in cols:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype("string")
                .str.strip()
                .str.upper()
            )
    return df


def fct_lower_string_columns(df: pd.DataFrame, cols: list = None) -> pd.DataFrame:
    """
    Objectif :
        Supprimer les espaces superflus et mettre toutes les lettres en minuscules,
        uniquement pour les valeurs de type str.
    
    Paramètres :
        df (pd.DataFrame) : DataFrame d'entrée.
        cols (list, optionnel) : Liste des colonnes à nettoyer. 
    Retour :
        pd.DataFrame : DataFrame avec les colonnes de chaînes nettoyées.
    """
    # Vérifier que des colonnes ont été fournies
    if not cols:
        print("[INFO] Aucune colonne spécifiée pour clean_string_columns_lower. Le DataFrame reste inchangé.")
        return df
    
    # Traiter chaque colonne spécifiée
    for col in cols:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype("string")
                .str.strip()
                .str.lower()
            )
    return df



def fct_iso_to_yyyymmddhhmmss(df: pd.DataFrame, col: str, new_col: str = None) -> pd.DataFrame:
    """
    Convertir une colonne ISO 8601 en une seule colonne YYYYMMDDhhmmss.
    Les valeurs manquantes ou invalides sont remplacées par :
        année=9999, mois=99, jour=99, heure=99, minute=99, seconde=99
    """
    if new_col is None:
        new_col = col

    def safe_convert(x):
        try:
            dt = pd.to_datetime(x, utc=True)
            year = f"{dt.year:04d}"
            month = f"{dt.month:02d}"
            day = f"{dt.day:02d}"
            hour = f"{dt.hour:02d}"
            minute = f"{dt.minute:02d}"
            second = f"{dt.second:02d}"
        except Exception:
            year = "9999"
            month = day = hour = minute = second = "99"

        return f"{year}{month}{day}{hour}{minute}{second}"

    df[new_col] = df[col].apply(safe_convert)
    return df



def fct_extract_edition(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    Extraire l'année d'une colonne de date ISO 8601.
    
    Paramètres :
        df (pd.DataFrame) : DataFrame d'entrée
        col (str) : colonne ISO à extraire l'année
    
    Retour :
        pd.DataFrame : DataFrame avec nouvelle colonne année
    """
    df['edition'] = df[col].str[:4]
    return df


In [3]:
#Récupération de la configuration
config_path = os.path.join(Path.cwd().parent, 'config.yaml')
config = fct_load_config(config_path)

# Transform

### Stadiums

In [4]:
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Exploration -------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Nombres des stades : 
print(f"\nNombre des stades : {df_stadiums['id'].nunique()}")


# verification des doublons dans df_stadiums

# df_stadiums.groupby('id').size().loc[lambda x: x > 1]

number_duplicates = df_stadiums.duplicated(subset = ['id']).sum()
print(f"\nNombre des 'id' des stadiums dupliqués: {number_duplicates}")

#valeurs nulles 
print(f"\nValeurs nulles dans df_stadiums:\n{df_stadiums.isnull().sum()}")

#types des colonnes
print(f"\nTypes des colonnes dans df_stadiums:\n{df_stadiums.dtypes}")

# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Transformation ----------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------
# Filtrer les colonnes nécessaires
columns_to_keep = ['id', 'name', 'city']
df_stadiums_transformed = df_stadiums[columns_to_keep].copy()

# Mettre en le premier caractère en Majuscule pour les colonnes name et city & convertir en type string
df_stadiums_transformed = fct_capitalize_string_columns(df_stadiums_transformed, ['name', 'city'])

print(f'\ndf_stadiums_transformed.dtypes:\n{df_stadiums_transformed.dtypes}')
df_stadiums_transformed.head()


Nombre des stades : 12

Nombre des 'id' des stadiums dupliqués: 0

Valeurs nulles dans df_stadiums:
id       0
name     0
city     0
lat      0
lng      0
image    0
dtype: int64

Types des colonnes dans df_stadiums:
id         int64
name      object
city      object
lat      float64
lng      float64
image     object
dtype: object

df_stadiums_transformed.dtypes:
id               int64
name    string[python]
city    string[python]
dtype: object


,id,name,city
0,1,Luzhniki stadium,Moscow
1,2,Otkrytiye arena,Moscow
2,3,Krestovsky stadium,Saint petersburg
3,4,Kaliningrad stadium,Kaliningrad
4,5,Kazan arena,Kazan


### tvchannels

In [5]:
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Exploration -------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------
# verification des doublons dans df_channels

# Nombres des chaines télévisées : 
print(f"\nNombre des chaines télévisées : {df_tvchannels['id'].nunique()}")


# df_tvchannels.groupby('id').size().loc[lambda x: x > 1]

number_duplicates = df_tvchannels.duplicated(subset = ['id']).sum()
print(f"\nNombre des 'id' des channels dupliqués: {number_duplicates}")

#valeurs nulles 
print(f"\nValeurs nulles dans df_tvchannels:\n{df_tvchannels.isnull().sum()}")  
#types des colonnes
print(f"\nTypes des colonnes dans df_tvchannels:\n{df_tvchannels.dtypes}")

# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Transformation ----------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------
# Filtrer les colonnes nécessaires
columns_to_keep = ['id', 'name', 'country', 'lang']
df_tvchannels_transformed = df_tvchannels[columns_to_keep].copy()

# Mettre en le premier caractère en Majuscule pour la colonne country & convertir en type string
df_tvchannels_transformed = fct_capitalize_string_columns(df_tvchannels_transformed, ['country'])
# Eclater les listes dans la colonne 'lang' en plusieurs lignes
df_tvchannels_transformed = df_tvchannels_transformed.explode('lang').reset_index(drop=True)

# Mettre en Majuscule pour les colonnes name et lang & convertir en type string
df_tvchannels_transformed = fct_upper_string_columns(df_tvchannels_transformed, ['name', 'lang'])

print(f'\ndf_tvchannels_transformed.dtypes:\n{df_tvchannels_transformed.dtypes}')
df_tvchannels_transformed.head()



Nombre des chaines télévisées : 22

Nombre des 'id' des channels dupliqués: 0

Valeurs nulles dans df_tvchannels:
id         0
name       0
icon       0
country    0
iso2       0
lang       0
dtype: int64

Types des colonnes dans df_tvchannels:
id          int64
name       object
icon       object
country    object
iso2       object
lang       object
dtype: object

df_tvchannels_transformed.dtypes:
id                  int64
name       string[python]
country    string[python]
lang       string[python]
dtype: object


,id,name,country,lang
0,1,DR1,Denmark,DA
1,2,TV2,Denmark,DA
2,3,BBC UK,Uk,EN
3,4,ITV UK,Uk,EN
4,5,ITV4 UK,Uk,EN


### teams

In [6]:
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Exploration -------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Nombres des équipes : 
print(f"\nNombre des équipes : {df_teams['id'].nunique()}")

# verification des doublons dans df_teams
# df_teams.groupby('id').size().loc[lambda x: x > 1]

number_duplicates = df_teams.duplicated(subset = ['id']).sum()
print(f"\nNombre des 'id' des teams dupliqués: {number_duplicates}")

#valeurs nulles 
print(f"\nValeurs nulles dans df_teams:\n{df_teams.isnull().sum()}")  
#types des colonnes
print(f"\nTypes des colonnes dans df_teams:\n{df_teams.dtypes}")

# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Transformation ----------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Filtrer les colonnes nécessaires
columns_to_keep = ['id', 'name']
df_teams_transformed = df_teams[columns_to_keep].copy()

# Dataframe teams
df_teams_transformed = fct_capitalize_string_columns(df_teams_transformed, ['name'])

print(f'\ndf_teams_transformed.dtypes:\n{df_teams_transformed.dtypes}')
df_teams_transformed.head(48)


Nombre des équipes : 32

Nombre des 'id' des teams dupliqués: 0

Valeurs nulles dans df_teams:
id             0
name           0
fifaCode       0
iso2           0
flag           0
emoji          0
emojiString    0
dtype: int64

Types des colonnes dans df_teams:
id              int64
name           object
fifaCode       object
iso2           object
flag           object
emoji          object
emojiString    object
dtype: object

df_teams_transformed.dtypes:
id               int64
name    string[python]
dtype: object


,id,name
0,1,Russia
1,2,Saudi arabia
2,3,Egypt
3,4,Uruguay
4,5,Portugal
5,6,Spain
6,7,Morocco
7,8,Iran
8,9,France
9,10,Australia


### groups

In [7]:
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Exploration -------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Nombres des groupes : 
print(f"\nNombre des groupes : {df_groups['group_id'].nunique()}")

# verification des doublons dans df_groups

# df_groups.groupby('id').size().loc[lambda x: x > 1]

number_duplicates = df_groups.duplicated(subset = ['group_id']).sum()
print(f"\nNombre des 'group_id' des groups dupliqués: {number_duplicates}")

#valeurs nulles 
print(f"\nValeurs nulles dans df_groups:\n{df_groups.isnull().sum()}")  
#types des colonnes
print(f"\nTypes des colonnes dans df_groups:\n{df_groups.dtypes}")

# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Transformation ----------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Dataframe groupes
df_groups_transformed = fct_upper_string_columns(df_groups, ['group_name'])
df_groups_transformed = fct_lower_string_columns(df_groups_transformed, ['group_id'])

print(f'\ndf_groups_transformed.dtypes:\n{df_groups_transformed.dtypes}')
df_groups_transformed.head(10)



Nombre des groupes : 8

Nombre des 'group_id' des groups dupliqués: 0

Valeurs nulles dans df_groups:
group_id            0
group_name          0
winner_team_id      0
runnerup_team_id    0
dtype: int64

Types des colonnes dans df_groups:
group_id            object
group_name          object
winner_team_id       int64
runnerup_team_id     int64
dtype: object

df_groups_transformed.dtypes:
group_id            string[python]
group_name          string[python]
winner_team_id               int64
runnerup_team_id             int64
dtype: object


,group_id,group_name,winner_team_id,runnerup_team_id
0,a,GROUP A,4,1
1,b,GROUP B,6,5
2,c,GROUP C,9,12
3,d,GROUP D,15,13
4,e,GROUP E,17,18
5,f,GROUP F,23,22
6,g,GROUP G,25,28
7,h,GROUP H,31,32


### rounds

In [8]:
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Exploration -------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Nombres des phases : 
print(f"\nNombre des phases : {df_rounds['round_id'].nunique()}")

# verification des doublons dans df_rounds

# df_rounds.groupby('round_id').size().loc[lambda x: x > 1]

number_duplicates = df_rounds.duplicated(subset = ['round_id']).sum()
print(f"\nNombre des 'round_id' des rounds dupliqués: {number_duplicates}")

#valeurs nulles 
print(f"\nValeurs nulles dans df_rounds:\n{df_rounds.isnull().sum()}")  
#types des colonnes
print(f"\nTypes des colonnes dans df_rounds:\n{df_rounds.dtypes}")
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Transformation ----------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

df_rounds_transformed = fct_lower_string_columns(df_rounds, ['round_id'])
df_rounds_transformed = fct_capitalize_string_columns(df_rounds_transformed, ['round_name'])

print(f'\ndf_rounds_transformed.dtypes:\n{df_rounds_transformed.dtypes}')
df_rounds_transformed.head()



Nombre des phases : 5

Nombre des 'round_id' des rounds dupliqués: 0

Valeurs nulles dans df_rounds:
round_id      0
round_name    0
dtype: int64

Types des colonnes dans df_rounds:
round_id      object
round_name    object
dtype: object

df_rounds_transformed.dtypes:
round_id      string[python]
round_name    string[python]
dtype: object


,round_id,round_name
0,round_16,Round of 16
1,round_8,Quarter-finals
2,round_4,Semi-finals
3,round_2_loser,Third place play-off
4,round_2,Final


### matches

In [9]:
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Exploration -------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Nombres des matches : 
print(f"\nNombre des matches : {df_matches['match_id'].nunique()}")

# Valeur stage : 
print(f"\nstage : {df_matches['stage'].unique().tolist()}")

# Valeur type : 
print(f"\ntype : {df_matches['type'].unique().tolist()}")

# ids des groups : 
print(f"\ngroup_id : {df_matches['group_id'].unique().tolist()}")

# ids des phases : 
print(f"\nround_id : {df_matches['round_id'].unique().tolist()}")

# verification des doublons dans df_matches

# df_matches.groupby('match_id').size().loc[lambda x: x > 1]

number_duplicates = df_matches.duplicated(subset = ['match_id']).sum()
print(f"\nNombre des 'match_id' des matches dupliqués: {number_duplicates}")

#valeurs nulles 
print(f"\nValeurs nulles dans df_matches:\n{df_matches.isnull().sum()}")  
#types des colonnes
print(f"\nTypes des colonnes dans df_matches:\n{df_matches.dtypes}")
# --------------------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------Transformation ----------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------------------------------

# Filtrer les colonnes nécessaires
columns_to_keep = ['match_id',
                   'stage' ,
                   'type',
                   'group_id',
                   'round_id',
                   'date',
                   'stadium_id',
                   'home_team_id',
                   'away_team_id',
                   'home_result',
                   'away_result',
                   'home_penalty',
                   'away_penalty',
                   'winner',
                   'matchday',
                   'channels']   
df_matches_transformed = df_matches[columns_to_keep].copy()
df_matches_transformed = fct_lower_string_columns(df_matches_transformed, ['group_id','round_id'])
df_matches_transformed = fct_capitalize_string_columns(df_matches_transformed, ['stage','type'])
df_matches_transformed['round_id'] = df_matches_transformed['round_id'].fillna('notdefined')
df_matches_transformed['group_id'] = df_matches_transformed['group_id'].fillna('notdefined')
                       

df_matches_transformed['edition'] = fct_extract_edition(df_matches_transformed, 'date')['edition']

df_matches_transformed = fct_iso_to_yyyymmddhhmmss(df_matches_transformed, 'date', 'formatted_date')

df_matches_transformed = df_matches_transformed.explode('channels').reset_index(drop=True)


# df_matches_transformed[df_matches_transformed['date'].isna()][['date', 'formatted_date']]
print(f'\ndf_rounds_transformed.dtypes:\n{df_rounds_transformed.dtypes}')
df_matches_transformed.head()


Nombre des matches : 64

stage : ['group', 'knockout']

type : ['group', 'qualified', 'winner', 'loser']

group_id : ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', None]

round_id : [None, 'round_16', 'round_8', 'round_4', 'round_2_loser', 'round_2']

Nombre des 'match_id' des matches dupliqués: 0

Valeurs nulles dans df_matches:
match_id         0
type             0
stage            0
group_id        16
round_id        48
date             0
stadium_id       0
home_team_id     0
away_team_id     0
home_result      0
away_result      0
home_penalty    60
away_penalty    60
winner          48
finished         0
matchday         0
channels         0
dtype: int64

Types des colonnes dans df_matches:
match_id          int64
type             object
stage            object
group_id         object
round_id         object
date             object
stadium_id        int64
home_team_id      int64
away_team_id      int64
home_result       int64
away_result       int64
home_penalty    float64
away_penalty

,match_id,stage,type,group_id,round_id,date,stadium_id,home_team_id,away_team_id,home_result,away_result,home_penalty,away_penalty,winner,matchday,channels,edition,formatted_date
0,1,Group,Group,a,notdefined,2018-06-14T18:00:00+03:00,1,1,2,5,0,NaN,NaN,NaN,1,4,2018,20180614150000
1,1,Group,Group,a,notdefined,2018-06-14T18:00:00+03:00,1,1,2,5,0,NaN,NaN,NaN,1,6,2018,20180614150000
2,1,Group,Group,a,notdefined,2018-06-14T18:00:00+03:00,1,1,2,5,0,NaN,NaN,NaN,1,13,2018,20180614150000
3,1,Group,Group,a,notdefined,2018-06-14T18:00:00+03:00,1,1,2,5,0,NaN,NaN,NaN,1,17,2018,20180614150000
4,1,Group,Group,a,notdefined,2018-06-14T18:00:00+03:00,1,1,2,5,0,NaN,NaN,NaN,1,20,2018,20180614150000


In [10]:
df_matches.head(64)


,match_id,type,stage,group_id,round_id,date,stadium_id,home_team_id,away_team_id,home_result,away_result,home_penalty,away_penalty,winner,finished,matchday,channels
0,1,group,group,a,None,2018-06-14T18:00:00+03:00,1,1,2,5,0,NaN,NaN,NaN,True,1,"[4, 6, 13, 17, 20, 22]"
1,2,group,group,a,None,2018-06-15T17:00:00+05:00,12,3,4,0,1,NaN,NaN,NaN,True,1,"[3, 6, 14, 17, 20, 22]"
2,17,group,group,a,None,2018-06-19T21:00:00+03:00,3,1,3,3,1,NaN,NaN,NaN,True,2,"[3, 6, 13, 17, 15, 21, 22]"
3,18,group,group,a,None,2018-06-20T18:00:00+03:00,10,4,2,1,0,NaN,NaN,NaN,True,2,"[3, 6, 13, 17, 21, 22]"
4,33,group,group,a,None,2018-06-25T18:00:00+04:00,7,4,1,3,0,NaN,NaN,NaN,True,3,"[4, 6, 13, 18, 15, 20, 22]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,60,winner,knockout,None,round_8,2018-07-07T18:00:00+04:00,7,23,28,0,2,NaN,NaN,28.0,True,5,"[4, 13, 16]"
60,61,winner,knockout,None,round_4,2018-07-10T21:00:00+03:00,3,9,25,1,0,NaN,NaN,9.0,True,6,"[4, 13, 16]"
61,62,winner,knockout,None,round_4,2018-07-11T21:00:00+03:00,1,15,28,2,1,NaN,NaN,15.0,True,6,"[3, 13, 15]"
62,63,loser,knockout,None,round_2_loser,2018-07-14T17:00:00+03:00,3,25,28,2,0,NaN,NaN,25.0,True,7,"[4, 13, 15]"


In [ ]:
print(f"\nValeurs nulles dans df_matches:\n{df_matches.isnull().sum()}")  


Valeurs nulles dans df_matches:
match_id         0
type             0
stage            0
group_id        16
round_id        48
date             0
stadium_id       0
home_team_id     0
away_team_id     0
home_result      0
away_result      0
home_penalty    60
away_penalty    60
winner          48
finished         0
matchday         0
channels         0
dtype: int64
